# Import Data

In [11]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load
school_file = "../UTAMCB201904DATA3/04-Pandas/Homework/Instructions/PyCitySchools/Resources/schools_complete.csv"
student_file = "../UTAMCB201904DATA3/04-Pandas/Homework/Instructions/PyCitySchools/Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_file)
student_data = pd.read_csv(student_file)

# Combine the data into a single dataset
mydata = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# Calc and Add Per Student Budget column
mydata["Per Student Budget"] = mydata["budget"]/mydata["size"]


# District Summary

In [12]:
# Calc District Summary
TtlSchoolCnt = school_data["school_name"].count()
TtlStudentsCnt = student_data["student_name"].count()
TtlBudget = school_data["budget"].sum()
AvgMathScore = student_data["math_score"].mean()
AvgReadScore = student_data["reading_score"].mean()
PcntPassMath = (student_data[student_data.math_score >= 70]["math_score"].count() /TtlStudentsCnt) *100
PcntPassReading = (student_data[student_data.reading_score >= 70]["reading_score"].count() /TtlStudentsCnt) *100
OverallPassRate = (PcntPassMath + PcntPassReading)/2

# Create a dataframe with results
TtlResults = pd.DataFrame({
    "Total Schools": [TtlSchoolCnt],
    "Total Students": [TtlStudentsCnt],
    "Total Budget": [TtlBudget],
    "Average Math Score": [AvgMathScore],
    "Average Reading Score": [AvgReadScore],
    "% Passing Math": [PcntPassMath],
    "% Passing Reading": [PcntPassReading],
    "% Overall Passing Rate": [OverallPassRate]
})

# Format columns
TtlResults["Total Students"] = TtlResults["Total Students"].map("{:,}".format)
TtlResults["Total Budget"] = TtlResults["Total Budget"].map("${:,.2f}".format)
TtlResults

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


# School Summary

In [13]:
# Calc School Summary
# [Dateframe 1] Group data by school name and get count, averages etc
schSummary = mydata.groupby(
   ['school_name']
).agg(
    {
         'Student ID':'count',
         'budget': 'first',
         'reading_score': 'mean',
         'math_score': 'mean',
         'Per Student Budget': 'first'
    }
)

# Format columns
schSummary["Per Student Budget"] = schSummary["Per Student Budget"].map("${:,.2f}".format)
schSummary["budget"] = schSummary["budget"].map("${:,.2f}".format)

# [Series 1] Calc Percent passed in math and create a series with school name and percent passed
schMathPass = mydata[mydata['math_score']>=70].groupby(['school_name']).count()
schMathPass = schMathPass["math_score"]

# [Series 2] Calc Percent passed in reading and create a series with school name and percent passed
schReadPass = mydata[mydata['reading_score']>=70].groupby(['school_name']).count()
schReadPass = schReadPass["reading_score"]

# Combine [Dataframe 1], [Series 1] and [Series 2] into one Dataframe
SummyAndMath = pd.merge(schSummary, schMathPass, on='school_name')
SummyAndMathAndRead = pd.merge(SummyAndMath, schReadPass, on='school_name')
SummyAndMathAndRead['math_score_y'] = (SummyAndMathAndRead["math_score_y"]/SummyAndMathAndRead["Student ID"]) *100
SummyAndMathAndRead['reading_score_y'] = (SummyAndMathAndRead["reading_score_y"]/SummyAndMathAndRead["Student ID"]) * 100

# Rename columns
SummyAndMathAndRead.columns = ['Total Students','Total School Budget','Average Reading Score',
                     'Average Math Score','Per Student Budget','% Passing Math', '% Passing Reading']

# Calc overall pass rate
SummyAndMathAndRead['% Overall Passing Rate'] = (SummyAndMathAndRead['% Passing Math'] + SummyAndMathAndRead['% Passing Reading'])/2

# Create a data frame with schoolname and type and then add type to results dataframe
schtype = school_data[['school_name','type']]
schSummyResults = pd.merge(SummyAndMathAndRead, schtype, on='school_name')

# Set school_name as index
schSummyResults = schSummyResults.set_index('school_name')

# Rename column
schSummyResults = schSummyResults.rename(columns={'type':'School Type'})

# Arrange columns
schSummyResults = schSummyResults[['School Type','Total Students','Total School Budget','Average Reading Score',
                     'Average Math Score','Per Student Budget','% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
# Remove index name
schSummyResults.index.name=""


# Top Performing Schools (By Overall Passing Rate)

In [14]:
#Top Performing Schools (By Passing Rate)
schSummyResults.sort_values(by='% Overall Passing Rate', ascending=False)[:5]

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",83.975780,83.061895,$582.00,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",83.848930,83.418349,$638.00,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",84.044699,83.839917,$609.00,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",83.816757,83.351499,$625.00,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",83.989488,83.274201,$578.00,93.867718,96.539641,95.203679


# Bottom Performing Schools (By Overall Passing Rate)

In [15]:
# Bottom Performing Schools (By Passing Rate)
schSummyResults.sort_values(by='% Overall Passing Rate', ascending=True)[:5]

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",80.744686,76.842711,$637.00,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",81.158020,76.711767,$639.00,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",81.182722,76.629414,$655.00,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",80.966394,77.072464,$650.00,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",80.746258,77.102592,$644.00,68.309602,79.299014,73.804308


# Math Scores by Grade

In [16]:
# Group by School Name and Grage and calculate math score average
mathSummary = student_data.groupby(['school_name','grade']).agg({'math_score':'mean'})

# Change rows into columns
mathSummary = mathSummary.unstack()

# Convert to Dataframe
mathSummary = pd.DataFrame(mathSummary.to_records())

# Set index
mathSummary = mathSummary.set_index("school_name")

# Empty index name
mathSummary.index.name=""

# Rename columns
mathSummary = mathSummary.rename(columns={
    "('math_score', '10th')":'10th',
    "('math_score', '11th')":'11th',
    "('math_score', '12th')":'12th',
    "('math_score', '9th')":'9th'
})

# Rearrange columns
mathSummary = mathSummary[['9th','10th','11th', '12th']]
mathSummary

,9th,10th,11th,12th
,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


# Reading Score by Grade

In [17]:
# Group by School Name and Grage and calculate reading score average
readSummary = student_data.groupby(['school_name','grade']).agg({'reading_score':'mean'})

# Change rows into columns
readSummary = readSummary.unstack()

# Convert to Dataframe
readSummary = pd.DataFrame(readSummary.to_records())

# Set index
readSummary = readSummary.set_index("school_name")

# Empty index name
readSummary.index.name=""

# Rename columns
readSummary = readSummary.rename(columns={
    "('reading_score', '10th')":'10th',
    "('reading_score', '11th')":'11th',
    "('reading_score', '12th')":'12th',
    "('reading_score', '9th')":'9th'
})

# Rearrange columns
readSummary = readSummary[['9th','10th','11th', '12th']]
readSummary

,9th,10th,11th,12th
,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


# Scores by School Spending

In [18]:
# Create bins and group names
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

# Add bin colum to the dataset
mydata["Bins"] = pd.cut(mydata["Per Student Budget"], spending_bins, labels=group_names)

# [Dataframe 1]Group by bin and calculate metrics
binSummary = mydata.groupby('Bins').agg({
    'Student ID':'count',
    'math_score':'mean',
    'reading_score':'mean'
})

# [Series 1] Calc Percent passed in math and create a series with school name and percent passed
binschMathPass = mydata[mydata['math_score']>=70].groupby(['Bins']).count()
binschMathPass = binschMathPass["math_score"]

# [Series 1] Calc Percent passed in math and create a series with school name and percent passed
binschReadPass = mydata[mydata['reading_score']>=70].groupby(['Bins']).count()
binschReadPass = binschReadPass["reading_score"]

# Combine [Dataframe 1], [Series 1] and [Series 2] into one Dataframe
binresults = pd.merge(binSummary, binschMathPass, on='Bins')
binSummyResults = pd.merge(binresults, binschReadPass, on='Bins')
binSummyResults['math_score_y'] = (binSummyResults["math_score_y"]/binSummyResults["Student ID"]) *100
binSummyResults['reading_score_y'] = (binSummyResults["reading_score_y"]/binSummyResults["Student ID"]) * 100

# Rename columns and calculate overall pass rate
binSummyResults.columns = ['Student ID','Average Math Score',
                     'Average Reading Score','% Passing Math', '% Passing Reading']
binSummyResults['% Overall Passing Rate'] = (binSummyResults['% Passing Math'] + binSummyResults['% Passing Reading'])/2

# Set index name
binSummyResults.index.name="Spending Ranges (Per Student)"

# Rearrange columns
binSummyResults = binSummyResults[['Average Math Score','Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
binSummyResults


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.363065,83.964039,93.702889,96.686558,95.194724
$585-615,83.529196,83.838414,94.124128,95.886889,95.005509
$615-645,78.061635,81.434088,71.400428,83.614770,77.507599
$645-675,77.049297,81.005604,66.230813,81.109397,73.670105


# Scores by School Size

In [19]:
# Create bins and groups
size_bins = [0, 1000, 2000, 5000]
size_group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Add Size Bins column to the dataset
mydata["Size Bins"] = pd.cut(mydata["size"], size_bins, labels=size_group_names)

# [Dataframe 1] Group by Size Bins and calculated metrics
sizebinSummary = mydata.groupby('Size Bins').agg({
    'Student ID':'count',
    'math_score':'mean',
    'reading_score':'mean'
})

# [Series 1] Calc Percent passed in math and create a series with school name and percent passed
sizebinschMathPass = mydata[mydata['math_score']>=70].groupby(['Size Bins']).count()
sizebinschMathPass = sizebinschMathPass["math_score"]

# [Series 2] Calc Percent passed in reading and create a series with school name and percent passed
sizebinschReadPass = mydata[mydata['reading_score']>=70].groupby(['Size Bins']).count()
sizebinschReadPass = sizebinschReadPass["reading_score"]

# Combine [Dataframe 1], [Series 1] and [Series 2] into one Dataframe
sizebinpreResults = pd.merge(sizebinSummary, sizebinschMathPass, on='Size Bins')
sizebinResults = pd.merge(sizebinpreResults, sizebinschReadPass, on='Size Bins')
sizebinResults['math_score_y'] = (sizebinResults["math_score_y"]/sizebinResults["Student ID"]) *100
sizebinResults['reading_score_y'] = (sizebinResults["reading_score_y"]/sizebinResults["Student ID"]) * 100

# Rename columns and create overall pass rate
sizebinResults.columns = ['Student ID','Average Math Score',
                     'Average Reading Score','% Passing Math', '% Passing Reading']
sizebinResults['% Overall Passing Rate'] = (sizebinResults['% Passing Math'] + sizebinResults['% Passing Reading'])/2

# Set index name
sizebinResults.index.name="School Size"

# Rearrange columns
sizebinResults = sizebinResults[['Average Math Score','Average Reading Score','% Passing Math', '% Passing Reading', '% Overall Passing Rate']]
sizebinResults

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,94.996400
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,95.194790
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,75.388769


# Scores by School Type

In [20]:
# [Dataframe 1] Group by School Type and calculated metrics
schTypeSummary = mydata.groupby(
   ['type']
).agg(
    {
         'Student ID':'count',
         'math_score': 'mean',
         'reading_score': 'mean'        
    }
)

# [Series 1] Calc Percent passed in math and create a series with school name and percent passed
schTypeMathPass = mydata[mydata['math_score']>=70].groupby(['type']).count()
schTypeMathPass = schTypeMathPass["math_score"]

# [Series 2] Calc Percent passed in reading and create a series with school name and percent passed
schTypeReadPass = mydata[mydata['reading_score']>=70].groupby(['type']).count()
schTypeReadPass = schTypeReadPass["reading_score"]

# Combine [Dataframe 1], [Series 1] and [Series 2] into one Dataframe
schTypepreResults = pd.merge(schTypeSummary, schTypeMathPass, on='type')
schTypeResults = pd.merge(schTypepreResults, schTypeReadPass, on='type')
schTypeResults['math_score_y'] = (schTypeResults["math_score_y"]/schTypeResults["Student ID"]) *100
schTypeResults['reading_score_y'] = (schTypeResults["reading_score_y"]/schTypeResults["Student ID"]) * 100

# Rename columns
schTypeResults.columns = ['Total Students','Average Math Score',
                     'Average Reading Score','% Passing Math', '% Passing Reading']

# Add overall Pass Rate
schTypeResults['% Overall Passing Rate'] = (schTypeResults['% Passing Math'] + schTypeResults['% Passing Reading'])/2

# Reorder columns
schTypeResults=schTypeResults[['Average Math Score','Average Reading Score','% Passing Math', 
                            '% Passing Reading', '% Overall Passing Rate']]
# Set index name
schTypeResults.index.name="School Type"
schTypeResults

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,95.173856
District,76.987026,80.962485,66.518387,80.905249,73.711818
